# Following the Money
In this notebook, I will look at what factors affect the compensation of an employee. I will mainly look at the effect of education, company information, experience, gender and country on the compensation.

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd

import os
from os import path

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display 

In [ ]:
file = '/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv'

In [ ]:
df = pd.read_csv(file)

First, I will give more meaningful names to the columns.

In [ ]:
df = df.rename(columns={"Q1": "Age", "Q2": "Gender", "Q3": "Residence", "Q4": "Education", 
                        "Q5": "Title", "Q6": "Coding Experience", 
                        "Q8": "Recommended PL", "Q11": "Computing Platform", 
                        "Q13": "TPU Usage", "Q15": "ML Experience", "Q20": "Company Size", 
                        "Q21": "ML Colleagues", "Q22": "ML Use At Company", "Q24": "Salary", 
                        "Q25": "ML Money Spent", "Q30": "Most Often Big Data Product",
                        "Q32": "Most Often Intelligence Tool", 
                        "Q38": "Primary Data Analysis Tool"})

In [ ]:
def renameCols(df, newName, maxIndex, question):
  dic = {}
  for i in range(1, maxIndex+1):
    colName = 'Q' + question + '_Part_' + str(i) 
    name = newName + str(i)
    dic[colName] = name
  colName = 'Q' + question + '_OTHER'
  dic[colName] = newName + '_Other'
  return df.rename(columns=dic)
  

In [ ]:
colNames = [ ('PL', 12, '7'), ('IDE', 11, '9'), ('Notebook', 13, '10'), 
            ('Hardware', 3, '12'), ('VisualLib', 11, '14'), ('ML Framework', 15, '16'), 
            ('ML Algo', 11, '17'), ('CV Method', 6, '18'), ('NLP Method', 5, '19'), 
            ('Work Activity', 7, '23'), ('Cloud Platform', 11, '26_A'), 
            ('Cloud Product', 11, '27_A'), ('ML Product', 10, '28_A'), 
            ('Big Data Product', 17, '29_A'), ('Intelligence Tool', 14, '31_A'), 
            ('AutoML Method', 7, '33_A'), ('AutoML Tool', 11, '34_A'), 
            ('Experiment Tool', 10, '35_A'), ('Sharing Platform', 9, '36'), 
            ('Course', 11, '37'), ('Media Source', 11, '39'), 
            ('Future Cloud Platform', 11, '26_B'), ('Future Cloud Product', 11, '27_B'), 
            ('Future ML Product', 10, '28_B'), ('Future Big Data Product', 17, '29_B'), 
            ('Future Intelligence Tool', 14, '31_B'), ('Future AutoML Method', 7, '33_B'), 
            ('Future AutoML Tool', 11, '34_B'), ('Future Experiment Tool', 10, '35_B')]

In [ ]:
for col in colNames:
    df = renameCols(df, col[0], col[1], col[2])

Then, I will drop the first row that includes the question descriptions since we won't be needing them.

In [ ]:
df = df.drop(0)


I will also define some additinal fields for later use. I will convert some of the categorical variables such as salary, coding experience, company size and ML experience to numeric. I will use the lower ends of the ranges as the numeric ranks. These variables are ordinal in nature so the numeric values we'll get will not be completely correct but they will give us good approximations in general.



In [ ]:
df.loc[df['Coding Experience'] == 'I have never written code', 'Coding Experience'] = '0 years'
df.loc[df['Coding Experience'] == '< 1 years', 'Coding Experience'] = '0.5 - 1 years'
df.loc[df['ML Experience'] == 'I do not use machine learning methods', 
       'ML Experience'] = '0 years'
df.loc[df['ML Experience'] == 'Under 1 year', 'ML Experience'] = '0.5 - 1 years'

df["Salary_Lower"] = df['Salary'].apply(
    lambda col: 
    str(col).split('-')[0].replace(',', '').replace('$', '').replace('>', '')).astype(float)
df['ML Colleagues_Lower'] = df['ML Colleagues'].apply(
    lambda col: 
    str(col).split('-')[0].replace('+', '')).astype(float)
df['Company Size_Lower'] = df['Company Size'].apply(
    lambda col: 
    str(col).split('-')[0].split()[0].replace(',', '')).astype(float)
df['Coding Experience_Lower'] = df['Coding Experience'].apply(
    lambda col: 
    str(col).split('-')[0].replace('<', '').split()[0].replace(
        ',', '').replace('+', '')).astype(float)
df['ML Experience_Lower'] = df['ML Experience'].apply(
    lambda col: str(col).split('-')[0].split()[0]).astype(float)
df['Age_Lower'] = df['Age'].apply(
    lambda col: str(col).split('-')[0].split()[0].replace('+', '')).astype(float)
df["ML Money Spent_Lower"] = df['ML Money Spent'].apply(
    lambda col: 
    str(col).split('-')[0].split()[0].replace(',', '').replace(
        '$', '').replace('>', '')).astype(float)

In [ ]:
df.loc[df['Education'] == 'I prefer not to answer', 'Education_Ordinal'] = np.NaN
df.loc[df['Education'] == 'No formal education past high school', 'Education_Ordinal'] = 0
df.loc[df['Education'] == 'Some college/university study without earning a bachelor’s degree', 
       'Education_Ordinal'] = 1
df.loc[df['Education'] == 'Bachelor’s degree', 'Education_Ordinal'] = 2
df.loc[df['Education'] == 'Professional degree', 'Education_Ordinal'] = 3
df.loc[df['Education'] == 'Master’s degree', 'Education_Ordinal'] = 3
df.loc[df['Education'] == 'Doctoral degree', 'Education_Ordinal'] = 4

In [ ]:
df.loc[df['ML Use At Company'] == 'I do not know', 'ML Use_Ordinal'] = np.NaN
df.loc[df['ML Use At Company'] == 'No (we do not use ML methods)', 'ML Use_Ordinal'] = 0
df.loc[df['ML Use At Company'] == 
       'We are exploring ML methods (and may one day put a model into production)', 
       'ML Use_Ordinal'] = 1
df.loc[df['ML Use At Company'] == 
       'We use ML methods for generating insights' + 
       ' (but do not put working models into production)', 
       'ML Use_Ordinal'] = 1
df.loc[df['ML Use At Company'] == 
       'We recently started using ML methods' + 
       ' (i.e., models in production for less than 2 years)', 
       'ML Use_Ordinal'] = 2
df.loc[df['ML Use At Company'] == 
       'We have well established ML methods' + 
       ' (i.e., models in production for more than 2 years)', 
       'ML Use_Ordinal'] = 3

Some few participants have more ML/Coding experience than their age so I will be selecting only the participants whose age is greater than their ML/Coding experience plus 14 years so someone with four years of ML experience would need to be at least at the age of 18. This might remove some very few valid participants as well but it is an easy way to filter out some pretty obvious anomalies.

In [ ]:
df = df[((df['Age_Lower'] > df['ML Experience_Lower'] + 14) & 
         (df['Age_Lower'] > df['Coding Experience_Lower'] + 14))]

In [ ]:
df.loc[df['Residence'] == 'South Korea', 'Residence'] = 'Republic of Korea'

I am not going to do much more data cleaning for now so let's just directly start with some visual representations of the data that could give us insights regarding the factors effecting the salary.

# Distribution of participants wrt. Salary

First, we will directly look at how the salary is distributed across the participants.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
graph = sns.countplot(ax=ax, x='Salary', data=df.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax.get_xticklabels(), rotation=30)

We see that the largest group earns less than 1,000\\$. Then, there is a slightly balanaced distribution up until 150,000$ after which the size of each group shrinks to less than 200 people.

One other thing to note here is that almost half of the data has missing salary information most of which are unemployed, college student or other.

In [ ]:
print('Total participants with no compensation: ', len(df.loc[df["Salary_Lower"].isna(), "Salary_Lower"]))
print('Students: ', len(df.loc[df['Title'] == 'Student']["Salary_Lower"].notna()))
print('Unemployed: ', len(df.loc[df['Title'] == 'Currently not employed']["Salary_Lower"].notna()))
print('Other: ', len(df.loc[df['Title'] == 'Other']["Salary_Lower"].notna()))

# Salary and Education

Then, we'll look at how education affects the salary. One normally assumes that with the increase in education level, there will be an increase in the salary compensation as well.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='Education', y='Salary_Lower', data=df.groupby(['Education']).mean().reset_index().sort_values(by='Education_Ordinal'))
g = graph.set_xticklabels(['No Education', 'Some College', 'Bachelor\'s', 'Master\'s', 'Professional', 'Doctoral', 'Prefer to Not Answer'], rotation=15)
ax1.set_title('Mean Salary Distribution wrt. Education')
graph = sns.countplot(ax=ax2, x='Salary', hue='Education', hue_order=['No formal education past high school', 
                                                                     'Some college/university study without earning a bachelor’s degree',
                                                                     'Bachelor’s degree', 'Master’s degree',  'Doctoral degree', 'Professional degree',
                                                                     'I prefer not to answer'], data=df.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and Education')
plt.show()

From the mean salary distribution, we see that there is an increase from bachelor's to master's and from master's to phd's. However, we also see that people with no university education earn as much as people with bachelor's degree and people with some college education earn slightly less than people with master's. Also, from the count distribution, we see that there are people with bachelor's, master's and doctoral degrees even among the group of 0-999$ earners. In fact, master's degree is dominant in all salary groups and there doesn't seem to be a clear cut correlation between education and salary. Of course, this results could depend on where exactly someone is working. So let's look at the two countries with the largest number of participants: India and USA.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
dfIndia = df.loc[df["Residence"] == "India"]
graph = sns.barplot(ax=ax1, x='Education', y='Salary_Lower', data=dfIndia.groupby(['Education']).mean().reset_index().sort_values(by='Education_Ordinal'))
g = graph.set_xticklabels(['No Education', 'Some College', 'Bachelor\'s', 'Master\'s', 'Professional', 'Doctoral', 'Prefer to Not Answer'], rotation=15)
ax1.set_title('Mean Salary Distribution wrt. Education for India')
graph = sns.countplot(ax=ax2, x='Salary', hue='Education', hue_order=['No formal education past high school', 
                                                                     'Some college/university study without earning a bachelor’s degree',
                                                                     'Bachelor’s degree', 'Master’s degree',  'Doctoral degree', 'Professional degree',
                                                                     'I prefer not to answer'], data=dfIndia.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and Education for India')
plt.legend(loc='upper right')
plt.show()


dfUSA = df.loc[df["Residence"] == "United States of America"]
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='Education', y='Salary_Lower', data=dfUSA.groupby(['Education']).mean().reset_index().sort_values(by='Education_Ordinal'))
g = graph.set_xticklabels(['No Education', 'Some College', 'Bachelor\'s', 'Master\'s', 'Professional', 'Doctoral', 'Prefer to Not Answer'], rotation=15)
ax1.set_title('Mean Salary Distribution wrt. Education for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='Education', hue_order=['No formal education past high school', 
                                                                     'Some college/university study without earning a bachelor’s degree',
                                                                     'Bachelor’s degree', 'Master’s degree',  'Doctoral degree', 'Professional degree',
                                                                     'I prefer not to answer'], data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and Education for USA')
plt.legend(loc='upper left')
plt.show()

As we have suspected, the compensation changes significantly depending on the country of employment, which is not very surprising. For India, the highest earners in terms of mean compensation appear to be people with master's degree while the phd's fall much below master's, bachelor's and even people with incomplete college education. In USA, however, phd's earn higher salaries than master's and bachelor's only falling short of people with professional degrees. 

Still, even if we just look at USA, we still see a somewhat unusual spike in the group of 0-999$ earners. At this point, one might think that these people (0-999\\$ group) are working in startup companies and they might not be getting compensation as an annual salary per se but they might own the business or have a stake in the company. We can look at the company size and number of colleagues for the employees from USA to check if these people are really working in small startups.

Since there aren't many people earning between 1,000 to 30,000 in USA. I will combine these groups to make the count graphs slightly smaller.

In [ ]:
dfUSA.loc[((dfUSA["Salary"] == "1,000-1,999") | (dfUSA["Salary"] == "2,000-2,999") | 
           (dfUSA["Salary"] == "3,000-3,999") | (dfUSA["Salary"] == "4,000-4,999") | 
           (dfUSA["Salary"] == "5,000-7,499") | (dfUSA["Salary"] == "7,500-9,999")  | 
           (dfUSA["Salary"] == "9,000-9,999")), "Salary"] = "1,000-9,999"
dfUSA.loc[((dfUSA["Salary"] == "10,000-14,999") | (dfUSA["Salary"] == "15,000-19,999")), 
          "Salary"] = "10,000-19,999"
dfUSA.loc[((dfUSA["Salary"] == "20,000-24,999") | (dfUSA["Salary"] == "25,000-29,999")), 
          "Salary"] = "20,000-29,999"

# Salary and Employment Information

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='Company Size', y='Salary_Lower', data=dfUSA.groupby(['Company Size']).mean().reset_index().sort_values(by='Company Size_Lower'))
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Mean Salary Distribution wrt. Company Size for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='Company Size', hue_order=['0-49 employees','50-249 employees','250-999 employees',  
                                                          '1000-9,999 employees',  '10,000 or more employees'], data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and Company Size for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='ML Colleagues', y='Salary_Lower', data=dfUSA.groupby(['ML Colleagues']).mean().reset_index().sort_values(by='ML Colleagues_Lower'))
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Mean Salary Distribution wrt. ML Colleagues for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='ML Colleagues', hue_order=['0','1-2','3-4', '5-9', '10-14', '15-19', '20+'], 
                      data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and ML Colleagues for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='ML Use At Company', y='Salary_Lower', data=dfUSA.groupby(['ML Use At Company']).mean().reset_index().sort_values(by='ML Use_Ordinal'))
g = graph.set_xticklabels(['No ML Use', 'Exploring ML methods', 'ML for insights','ML Methods in production < 2 years', 'ML methods in production for 2+ years', 'I do not know'], rotation=15)
ax1.set_title('Mean Salary Distribution wrt. ML Use At Company for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='ML Use At Company', data=dfUSA.sort_values(by='Salary_Lower'), hue_order=['I do not know', 'No (we do not use ML methods)',
                                                                              'We are exploring ML methods (and may one day put a model into production)',
                                                                              'We use ML methods for generating insights (but do not put working models into production)',
                                                                              'We recently started using ML methods (i.e., models in production for less than 2 years)',
                                                                              'We have well established ML methods (i.e., models in production for more than 2 years)'])
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and ML Use At Company for USA')
plt.show()

As the company size or # of ML Colleagues increases, we also see an increase in the mean compensation. The companies with ML algorithms in production tend to pay higher salaries on average than the companies without any ML algorithms in use as well. Also, for the group of 0-999\\$ earners, we see that an overwhelming majority of them works in companies with less than 50 employees, which might indicate that we were indeed right when we assumed these people are working at startup companies. Furthermore, we see that most of the people in 0-999\\$ group have less than three colleagues working on Machine Learning or Data Science related fields and the companies they are working at usually does not have ML methods in production, which again indicates that they are either working in a very small startup or they might be working for a company that does not rely on data science much. In any case, the spike at the 0-999\\$ group appears to be caused by the people working at small or non-ML related companies.

# Salary and Experience
Now, I will look at how coding and ML experience affects the compensation.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='ML Experience', y='Salary_Lower', data=dfUSA.groupby(['ML Experience']).mean().reset_index().sort_values(by='ML Experience_Lower'))
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Mean Salary Distribution wrt. ML Experience for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='ML Experience', hue_order=['0 years', "0.5 - 1 years", '1-2 years', '2-3 years', '3-4 years', '4-5 years', '5-10 years', 
                                                                          '10-20 years', '20 or more years'], data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and ML Experience for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='Coding Experience', y='Salary_Lower', data=dfUSA.groupby(['Coding Experience']).mean().reset_index().sort_values(by='Coding Experience_Lower'))
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Mean Salary Distribution wrt. Coding Experience for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='Coding Experience', hue_order=['0 years', '0.5 - 1 years', '1-2 years', '3-5 years', '5-10 years', '10-20 years', 
                                                                              '20+ years'], data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and Coding Experience for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=[20,5], ncols=2)
dfCount = dfUSA[dfUSA['Salary_Lower'] == 0].groupby(['Coding Experience', 'ML Experience']).count().reset_index()
dfCount = dfCount.pivot(index='Coding Experience', columns='ML Experience', values='Coding Experience_Lower')
dfCount.index = pd.CategoricalIndex(dfCount.index, categories= ["0.5 - 1 years", "1-2 years", "3-5 years", "5-10 years", "10-20 years", "20+ years"])
dfCount.sort_index(level=0, ascending=False, inplace=True)
dfCount.columns = pd.CategoricalIndex(dfCount.columns, categories= ['0 years', "0.5 - 1 years", "1-2 years", "2-3 years", "3-4 years", "4-5 years", "5-10 years", "10-20 years", "20 or more years"])
dfCount.sort_index(level=0, inplace=True, axis=1)
dfCount = dfCount / dfCount.sum().sum() * 100
sns.heatmap(dfCount, ax=ax1, annot=True, cmap='Reds')
a = ax1.set_title('ML Experience vs Coding Experience Map for 0-999$ earners as percentages')

dfCount = dfUSA.groupby(['Coding Experience', 'ML Experience']).count().reset_index()
dfCount = dfCount.pivot(index='Coding Experience', columns='ML Experience', values='Coding Experience_Lower')
dfCount.index = pd.CategoricalIndex(dfCount.index, categories= ["0.5 - 1 years", "1-2 years", "3-5 years", "5-10 years", "10-20 years", "20+ years"])
dfCount.sort_index(level=0, ascending=False, inplace=True)
dfCount.columns = pd.CategoricalIndex(dfCount.columns, categories= ['0 years', "0.5 - 1 years", "1-2 years", "2-3 years", "3-4 years", "4-5 years", "5-10 years", "10-20 years", "20 or more years"])
dfCount.sort_index(level=0, inplace=True, axis=1)
dfCount = dfCount / dfCount.sum().sum() * 100
sns.heatmap(dfCount, ax=ax2, annot=True, cmap='Reds')
a = ax2.set_title('ML Experience vs Coding Experience Map for all salary groups as percentages')
plt.show()

Not surprisingly, we see that more coding or ML experience leads to better mean compensation. Another thing interesting to me is that if we look at the group of 0-999\$ earners again, a majority of them seem to have 0 to 2 years of ML experience and more than 10 years of coding experience (there are some anomalies here though because if we look at the heatmap on the right, we see some people in the lower right corner. I hardly believe that anyone with 0-1 years of coding experience can have 5-10 years of ML experience. How can they do data science for more than 5 years without having almost no coding experience? But their percentage is very small that it doesn't make much difference for us here). 

Based on the heatmaps, 44\% of 0-999\\$ earners have more than 5 years of coding experience while having less than 4 years of ML experience as opposed to 32\% for all salary groups. Another 37\% of them have less than 5 years coding experience and less than 4 years of ML experience as opposed to 45\% for all salary groups. Finally, 18\% of 0-999\$ earners have more than 4 years of ML experience as opposed to 21% for all salary groups. 

We have already established that most of the 0-999\$ earners are working in small companies and have low number of colleagues. Now, we also see that almost half (44%\) of them are transitioning from other computer science fields to data science, which is 12\% higher than the usual result we get for all salary groups. In conclusion, these people seem to be highly risk tolerant.

# Salary and Money Spent on ML
Then, there is the money spent on ML. I would normally expect the people with higher income to spent more money on ML tools and softwares/hardwares.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(24, 8), ncols=2, gridspec_kw={'width_ratios': [2, 3]})
graph = sns.barplot(ax=ax1, x='ML Money Spent_Lower', y='Salary_Lower', data=dfUSA.groupby(['ML Money Spent_Lower']).mean().reset_index().sort_values(by='ML Money Spent_Lower'))
g = graph.set_xticklabels(['$0 ($USD)', '$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)'], rotation=30)
ax1.set_title('Mean Salary Distribution wrt. ML Money Spent for USA')
graph = sns.countplot(ax=ax2, x='Salary', hue='ML Money Spent', hue_order=['$0 ($USD)', '$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', 
                                                                           '$100,000 or more ($USD)'], data=dfUSA.sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Distribution of Participants wrt. Salary and ML Money Spent for USA')
plt.show()

pie, ax = plt.subplots(figsize=[12,6])
labels = ['$0 ($USD)', '$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '\$100,000 or more (\$USD)']
dfPie = dfUSA.groupby(['ML Money Spent_Lower']).count()['Age']
plt.pie(x=dfPie / dfPie.sum() * 100, autopct="%.1f%%", labels=labels)
t = plt.title("Distribution of ML Money Spent at home or work", fontsize=14)

Not many surprises here. We see that the mean salary increases as people spend more money on ML. Maybe the only thing interesting here is that more than 30\% of the participants said they spend 0 money on ML, which seems high. The question in the supplementary file says 'you (or your team) ... at home (or at work)' but some of them might have thought the question is asking only for home use.  

# Correlation of Salary and Other Variables
Finally, we'll look at the spearman rank correlation (since we have ordinal data). 

In [ ]:
fig, ax = plt.subplots(figsize=[10,5])
cor = dfUSA[['ML Colleagues_Lower', 'Company Size_Lower', 'Education_Ordinal', 'Coding Experience_Lower', 'ML Experience_Lower', 
                     'ML Use_Ordinal', 'ML Money Spent_Lower', 'Salary_Lower']].corr(method="spearman")
sns.heatmap(cor, annot=True, cmap='Reds')
a = ax.set_title('Spearman Correlation Map')
plt.show()

pValues = []
dfTemp = dfUSA[dfUSA['Salary_Lower'].notna()]
dfTemp2 = dfTemp[dfTemp['ML Colleagues_Lower'].notna()]
pValues.append(stats.pearsonr(dfTemp2['ML Colleagues_Lower'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['Company Size_Lower'].notna()]
pValues.append(stats.pearsonr(dfTemp2['Company Size_Lower'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['Education_Ordinal'].notna()]
pValues.append(stats.pearsonr(dfTemp2['Education_Ordinal'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['Coding Experience_Lower'].notna()]
pValues.append(stats.pearsonr(dfTemp2['Coding Experience_Lower'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['ML Experience_Lower'].notna()]
pValues.append(stats.pearsonr(dfTemp2['ML Experience_Lower'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['ML Use_Ordinal'].notna()]
pValues.append(stats.spearmanr(dfTemp2['ML Use_Ordinal'], dfTemp2['Salary_Lower'])[1])
dfTemp2 = dfTemp[dfTemp['ML Money Spent_Lower'].notna()]
pValues.append(stats.spearmanr(dfTemp2['ML Money Spent_Lower'], dfTemp2['Salary_Lower'])[1])
d = {'Variable Name': ['ML Colleagues_Lower', 'Company Size_Lower', 'Education_Ordinal', 'Coding Experience_Ordinal', 'ML Experience_Lower', 
                     'ML Use_Ordinal', 'ML Money Spent_Lower'], 'P-Value': pValues}
pDf = pd.DataFrame(data=d)
#display(pDf)

Salary is most correlated with ML experience closely followed by ML Use at the company, money spent on ML and the coding experience. Salary has a decent correlation with # of ML colleagues as well and it is also slightly correlated with education level and company size although it is not as much as the other five variables on the map. Still, the fact that ML colleagues and Company Size are correlated with salary tells us that the size of a company and the size of ML/Data Science department at a company might be a good reflector of how much compensation an employee receives. We also see that the companies that have well established ML methods in production seem to be better at compensating their employees due to the correlation between salary and ML use at work. 

Aside from the salary, the correlation map also shows us that ML use at work is correlated with money spent on ML, ML experience and # of ML Colleagues (i.e. if the company has ML methods in production, it is more likely that employees will have more ML colleagues, more ML experience and spent more money on ML). Of course, another thing we see is that company size and # of ML colleagues are highly correlated with one another, which is not very surprising since larger companies tend to have more workers to begin with. Lastly, we see that the company size seem to have little to no correlation with education level, coding experience and ML experience.

# Salary and Gender

Probably, this is the most asked question when it comes the compensation so here, we go.

In [ ]:
dfUSAGender = dfUSA[((dfUSA['Gender'] != 'Nonbinary') & (dfUSA['Gender'] != 'Prefer not to say') & (dfUSA['Gender'] != 'Prefer to self-describe'))]
dfUSAGender = dfUSAGender[dfUSAGender['Salary_Lower'] > 0]

fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='Gender', y='Salary_Lower', data=dfUSAGender, meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2))
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender for USA')
graph = sns.violinplot(ax=ax2, x='Gender', y='Salary_Lower', data=dfUSAGender)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender for USA')
plt.show()

I am not going to talk much about 'Non-binary' and 'Self-described' groups since those groups have less than 20 members each. Rather, we will just focus on man vs woman. I am removing the 0-999$ earners from the data as well since it lowers the salary of women too much in some cases. 

In the graphs, green line represent the mean and the black line is the median.

If we just look at the gender and nothing else, women get paid less than men on average but the median compensation for men and women are more or less the same. We can also see that upper whisker of the box plot and the upper end of the violin plot go higher for men than women, which means women have a much more balanced salary distribution while there are a significant number of men who earn much higher salaries than average salary for men.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='Education', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Education_Ordinal'), meanline=True, 
                    showmeans=True, meanprops=dict(linestyle='-', linewidth=2),  hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(['No Education', 'Some College', 'Bachelor\'s', 'Master\'s', 'Professional', 'Doctoral', 'Prefer to Not Answer'], rotation=15)
ax1.set_title('Salary Distribution wrt. Gender and Education for USA')
graph = sns.violinplot(ax=ax2, x='Education',  y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Education_Ordinal'), 
                       hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(['No Education', 'Some College', 'Bachelor\'s', 'Master\'s', 'Professional', 'Doctoral', 'Prefer to Not Answer'], rotation=15)
ax2.set_title('Kernel Density of Salary wrt. Gender and Education for USA')
plt.show() 

If we account for the education level though, women get paid less than men in terms of mean and median compensation at every education level where the group size is at least 20 or more for the women (Bachelor's, Master's and doctoral). We also see from the violin plot that men with some college education but no degree has a very imbalanced salary distribution compared to the women in the same category. Also, in general, distributions for men have higher variance than the women's.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='Age', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Age_Lower'),
                    meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2), hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender and Age for USA')
graph = sns.violinplot(ax=ax2, x='Age', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Age_Lower'), hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender and Age for USA')
plt.show()

We can also look at the effect of age. For people between ages 25-35, the mean and median salaries are close. After the age of 35, the pay gap widens and men typically earn more money than women in terms of median and mean compensation. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='Company Size', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Company Size_Lower'),
                    meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2), hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender and Company Size for USA')
graph = sns.violinplot(ax=ax2, x='Company Size', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Company Size_Lower'),
                       hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender and Company Size for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='ML Colleagues', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='ML Colleagues_Lower'),
                    meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2), hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender and ML Colleagues for USA')
graph = sns.violinplot(ax=ax2, x='ML Colleagues', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='ML Colleagues_Lower'), 
                       hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender and ML Coleagues for USA')
plt.show()

For company size, the biggest compensation discrepency is seen in the small companies (0-49 employees) where the median salary of women is equal to the 25th percentile of men whereas the smallest discrepancy is in the moderate sized companies (250-999 employees) where men and women have almost the same mean and median salaries. 

For number of ML colleagues, the gender pay gap is high for women with 10 or more ML colleagues whereas below 10 colleagues, the difference is small when it comes to mean and median compensation. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='Coding Experience', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Coding Experience_Lower'),
                    meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2), hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender and Coding Experience for USA')
graph = sns.violinplot(ax=ax2, x='Coding Experience', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='Coding Experience_Lower'),
                       hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender and Coding Experience for USA')
plt.show()

fig, (ax1, ax2) = plt.subplots(figsize=(16, 7), ncols=2)
graph = sns.boxplot(ax=ax1, x='ML Experience', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='ML Experience_Lower'),
                    meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2), hue_order=['Man', 'Woman'])
g = graph.set_xticklabels(ax1.get_xticklabels(), rotation=30)
ax1.set_title('Salary Distribution wrt. Gender and ML Experience for USA')
graph = sns.violinplot(ax=ax2, x='ML Experience', y='Salary_Lower', hue='Gender', data=dfUSAGender.sort_values(by='ML Experience_Lower'), 
                       hue_order=['Man', 'Woman'], split=True)
g = graph.set_xticklabels(ax2.get_xticklabels(), rotation=30)
ax2.set_title('Kernel Density of Salary wrt. Gender and ML Experience for USA')
plt.show()

Finally, we will look at the gender pay gap while accounting for the coding experience and ML experience. The median compensation seems to be relatively close for men and women but man still seem to be holding the upperhand for mean salary compensation in most categories of ML experience and Coding experience. Another important thing is that the whiskers of the box plots usually are longer in the case of men meaning that men are more prevalant in both upper and lower income groups than women, with the only exception of the group with 10-20 year ML experience, which is also the only group in which women slightly earn more than men in terms of mean salary and median. On the other hand, the violin plots are pretty identical for men and women when we account for the ML and coding experience at least compared to the results we had for age, education level, company size and ml colleagues.

In conclusion, when we account for the coding and ML experience, the median salaries of men and women are close in most of the groups. On the other hand, for the same level of experience, the salary of men have higher variance whereas the salary of women are much closer to one another (the fact that there are more male participants in the survey might be playing some role in this). The men usually get paid more on average because of the long upper whiskers their box plots have and they tend to have more outliers than women as well.

# Salary and Country
Now, we will get away from USA and look at the globe once again.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
graph = sns.barplot(ax=ax, x='Residence', y='Salary_Lower', data=df.groupby(['Residence']).median().reset_index().sort_values(by='Salary_Lower'))
g = graph.set_xticklabels(ax.get_xticklabels(), rotation=90)
a = ax.set_title('Median Salary Distribution wrt. Country')

In terms of median compensation, USA, Switzerland and Israel are leading the race. On the lower end of the spectrum, we see Nigeria, Tunisia and Pakistan. In general, more developed countries fall to the upper end of the spectrum while the undeveloped countries tend to be in the lower end, which is not very contrary to what one would expect. Now, I will group these countries into 6 different bins based on the mean compensation and then, generate box plots for each group.

In [ ]:
dfResidence = df.groupby(['Residence']).median().reset_index().sort_values(by='Salary_Lower')['Residence']
dfResList = [dfResidence[0:14], dfResidence[14:21], dfResidence[21:29], dfResidence[29:40], dfResidence[40:50], dfResidence[51:54]]
countryGroup = ['Very Low Income', 'Low Income', 'Low to Moderate', 'Moderate to High', 'High Income', 'Very High Income']
fig, ax = plt.subplots(figsize=(20, 36), ncols=2, nrows=3)
for i, dfRes in enumerate(dfResList):
  dfResidence1 = pd.merge(df, dfRes, how='inner', on=['Residence'])
  if i == 0:
    dfResidence1 = dfResidence1[dfResidence1['Salary_Lower'] < 50000]
  elif i == 1 or i == 2 or i == 3:
    dfResidence1 = dfResidence1[dfResidence1['Salary_Lower'] < 200000]
  elif i == 4:
    dfResidence1 = dfResidence1[dfResidence1['Salary_Lower'] < 300000]
  graph = sns.boxplot(ax=ax[int(i / 2)][int(i % 2)], x='Residence', y='Salary_Lower', data=dfResidence1, 
                      meanline=True, showmeans=True, meanprops=dict(linestyle='-', linewidth=2))
  g = graph.set_xticklabels(ax[int(i / 2)][int(i % 2)].get_xticklabels(), rotation=75)
  ax[int(i / 2)][int(i % 2)].set_title('Salary Distribution wrt. Country for Salary Group: ' + countryGroup[i])
fig.tight_layout()
plt.show()

In almost every country, there are people with very high incomes (300,000-500,000\\$) so to make the first four figures more readable, I have cut the people with income above 50,000\$ from the country group very low income and the people with income above 200,000\\$ from the country groups of low, low to moderate and moderate to high income.

The very low income group contains countries with median compensation lower than 5,000$. Most of these countries have many outliers in their graphs and they also show a high deviation between the median and mean salaries.

The low income group contains the countries with median compensation lower than 10,000\\$. One interesting country here is Saudi Arabia with an upper whisker ending at almost 100,000\\$ while also having a median compensation of only 7,500\$. It is arguably the country with the largest spread of compensation values in its box plot. We also see that India has many outliers in its graph.

The low to moderate income group contains countries with median compensation lower than 15,000$. There is not much to say for this group. These countries are very similar to group low income.

The moderate to high income group contains countries with median compensation lower than 40,000$. Many of these are european countries. The box plot for Belgium is a little bit surprising to see since it has a large spread similar to Saudi Arabia's despite having a higher median compensation.

The high income group contains countries with median compensation lower than 90,000$. I am a little bit surprised to see that Ireland has a higher median than Japan and France (and it is equivalent to sweden and singapore). Another interesting thing is that based on this calcuation, Ireland, United Arab Emirates and Canada are the only countries among all the countries to have mean salaries lower than their median salaries. 

Finally, the very high income group contains countries with median compensation above 90,000$. There are only three countries in this group. USA and Switzerland look almost identical to one another and Israel looks like a slightly pushed down version of the other two.

I also took the GDP per capita (PPP) data from https://www.kaggle.com/frankmollard/income-by-country/metadata. I am going to look at how median compensation for each country compares to the GDP per capita (PPP) of that country.

In [ ]:
def getLatestMetric(df, name):
  df = df.parse(name) 
  df = df[['Country', 2018]]
  df = df.rename(columns={'Country':'Residence'})
  df.loc[df['Residence'] == 'Iran (Islamic Republic of)', 'Residence'] = 'Iran, Islamic Republic of...'
  df.loc[df['Residence'] == 'Korea (Republic of)', 'Residence'] = 'Republic of Korea'
  df.loc[df['Residence'] == 'Russian Federation', 'Residence'] = 'Russia'
  df.loc[df['Residence'] == 'United Kingdom', 'Residence'] = 'United Kingdom of Great Britain and Northern Ireland'
  df.loc[df['Residence'] == 'United States', 'Residence'] = 'United States of America'
  return df

incomeFile = "/kaggle/input/income-by-country/Income by Country.xlsx"
xl = pd.ExcelFile(incomeFile)
dfGDP = getLatestMetric(xl, "GDP per capita")

In [ ]:
dfRes = df.groupby(['Residence']).median().reset_index()
dfGDPRelevant = pd.merge(dfGDP, dfRes, how='inner', on=['Residence'])
dfGDPRelevant['Median Salary to GDP per Capita Ratio'] = dfGDPRelevant['Salary_Lower'] / dfGDPRelevant[2018]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
graph = sns.barplot(ax=ax, x='Residence', y='Median Salary to GDP per Capita Ratio', data=dfGDPRelevant.sort_values(by='Median Salary to GDP per Capita Ratio'))
g = graph.set_xticklabels(ax.get_xticklabels(), rotation=90)
a = ax.set_title('')

The y-axis shows the ratio of the median compensation of the ML employees to GDP per capita. GDP per capita here serves as the average prosperity of an individual in a particular country. A ratio of 1 for a country means that the median compensation of the participants from that country in the survey is more or less the same as the GDP per capita of that country so we would expect the data science employees to have a relatively similar standard of living to an average citizen. A ratio higher than 1 would indicate that the country is providing a higher standard of living for its data science employees than the average citizen whereas a ratio below 1 would indicate that working in a data science related field might provide a worse standard of living than what GDP per capita shows. 

In the bar chart, we see many of the middle eastern / northern african countries such as Iran, Egypt, Morocco, Saudi Arabia and Turkey fall towards the lower end of the spectrum. These countries typically have a higher GDP per capita than countries like Kenya, Philipines and Ghana but their median compensation for the data science field is not much higher than the median compensation of the three aforementioned countries, which results in a very low median salary to GDP per capita ratio. In a way, I think that these countries have the option of paying their data science employees better since they seem to have high enough GDP per capita for that but they don't (maybe there might be some other regional factor affecting this result). On the upper end of the spectrum, we see USA, Israel and Switzerland again but we also see that Nepal and South Africa have higher median compensations than their GDP per capitas. In the middle part, Korea and Singapore seems to have surprisingly low ratios (around 0.5). Another thing to note is that close to 70\% of these countries have a ratio less than 1.

# Conclusion

There are many factors that affects the compensation of a ML/DS employee. ML experience and Coding experience usually is a good indicator of what salary one might expect but we also see that the structure of the company (company size, ML colleagues, ML use at work etc.) plays a good role in determining how much salary one could get as well. The education also has some effect on the compensation although its influence seems to be rather more limited than the other factors we have mentioned so far. 

Then, there is the question whether the gender affects the compensation or not. When we look at the gender pay gap, there is always a difference (most often in the favor of men) but how much difference we see higly relies on what factors we take into account when we are looking at the salary distributions. Accounting for coding and ML experiences seem to be giving us the smallest gap although there still is a gap whereas accounting for age or education results in larger gaps between men and women.

However, we also need to keep in mind that the salary analysis here mostly depends on the data from USA and we know that the compensation could vary significantly from one country to another. In fact, country of employment is the biggest factor affecting one's compensation and it also causes the largest gap among the employess from different countries. It would be interesting to analyze the other countries in a way similar to USA and see if any of the previously mentioned factors change significantly for any other country. Alas, I do not think that there is enough data here to do an in depth salary analysis of each and every country.